<a href="https://colab.research.google.com/github/utkarshminhas/Violence-Detection-FYP/blob/main/Jupyter%20Notebooks/primitive_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/utkarshminhas/Violence-Detection-FYP.git

Cloning into 'Violence-Detection-FYP'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 88 (delta 27), reused 65 (delta 13), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [29]:
from google.colab import drive
drive.mount('/content/drive')

!ls '/content/drive/Shareddrives/Final Year Project/Datasets'

data_dir = '/content/drive/Shareddrives/Final Year Project/Datasets/Mini-RWF-2000'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 HockeyFights	 Peliculas      RWF-2000
 Mini-RWF-2000	 Raw-RWF-2000  'Violent Flows Dataset'


In [23]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
# import keras_metrics as km
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import tqdm.notebook as tq
import tensorflow_addons as tfa

In [3]:
!ls '$data_dir'
img_height , img_width = 128, 128
seq_len = 1
classes = ["Fight", "Non Fight"]

Fight  NonFight


In [4]:
fgbg = cv2.createBackgroundSubtractorMOG2(
    varThreshold=15,
    detectShadows=True
)

def get_mask(frame):
    mog_mask = fgbg.apply(frame)
    median_blur_mask = cv2.medianBlur(mog_mask, 5)
    bilateral_filter_mask = cv2.bilateralFilter(median_blur_mask, 9, 75, 75)
    gaussian_blur_mask = cv2.GaussianBlur(bilateral_filter_mask, (13, 13), 5)

    return gaussian_blur_mask

In [5]:
# def _frames_extraction(video_path):
#     frames_list = []
     
#     vidObj = cv2.VideoCapture(video_path)
#     # Used as counter variable 
#     count = 1
 
#     while count <= seq_len: 
#         success, image = vidObj.read() 
#         if success:
#             image = cv2.resize(image, (img_height, img_width))
#             frames_list.append(image)
#             count += 1
#         else:
#             print("Defected frame")
#             break
            
#     return frames_list

def frames_extraction(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)

    frame_count = 1

    while True:
        success, frame = cap.read()

        if not success or frame is None:
            break

        # new_frame = get_mask(frame)
        # cv2.imshow('Novel Preprocessed Frame', new_frame)
        frames.append(frame)

        k = cv2.waitKey(30)
        if frame_count >= seq_len or k == 27 or k == ord('q'):
            break

        frame_count += 1

    cap.release()
    # cv2.destroyAllWindows()

    return frames

In [47]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        # print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in tq.tqdm(files_list):
            frames = frames_extraction(os.path.join(input_dir, c, f))
           
            if len(frames) == seq_len:
                X.append(frames)
                y = [0] * len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)

    X = np.expand_dims(X, -1);
    Y = np.expand_dims(Y, -1);


    return X, Y

In [49]:
X, Y = create_data(data_dir)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)


In [51]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation = "softmax"))

opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

earlystop = EarlyStopping(patience=7)
tqdm_callback = tfa.callbacks.TQDMProgressBar()
callbacks = [earlystop, tqdm_callback]

history = model.fit(x = X_train, y = y_train, epochs=1, shuffle=True, validation_split=0.2, callbacks=callbacks)
# ^^^ FIX THIIIIIS ^^^

# model.fit(X_train, y_train,
#           batch_size=64,
#           epochs=1,
#           verbose=0,
#           callbacks=callbacks,
#           validation_data=(X_test, y_test))

ValueError: ignored